## Preprocessing

In [1]:
user = ''
model_name = 'metal_prediction_CNN'

In [2]:
import sys
import json

print ("Initializing global variables...", end=' ')
sys.stdout.flush()

# Filepaths
output_file = './logs/results.txt'
hist_path = model_path = fig_path = './logs/'
dict_path = './dictionaries/'

print ("Done")
print ("  Filepath set to ./logs/")

##################################################

print ("Importing modules...", end=' ')
import modules
print ("Done")

##################################################

print ("Reading data from disk...", end=' ')
sys.stdout.flush()

import numpy as np
import pandas as pd

df = pd.read_parquet('./datasets/Metal_all_20180601.parquet')
seqs = np.array(df.sequence)
target = np.array(df.ligandId)
cluster_numbers = np.array(df.clusterNumber90)

print (set(target))

# FOFE
label_dict = {}
with open(dict_path + "metal_dict", 'r') as fp:
        label_dict = json.load(fp)

for i in range(target.shape[0]):
    target[i] = [label_dict[target[i]]]

# with open(dict_path + "metal_dict", 'w') as fp:
#     json.dump(label_dict, fp)    

print ("Done")

##################################################

print ("Loading dictionaries...", end=' ')
sys.stdout.flush()

# FOFE
vocab_dic_fofe = {}
with open(dict_path + "vocab_dict_fofe", 'r') as fp:
        vocab_dic_fofe = json.load(fp)

print ("Done")

##################################################

print ("Performing cross validation split...", end=' ')
ratio = 0.9
split = int(ratio*len(seqs))
train_seqs, val_seqs = seqs[:split], seqs[split:]
train_label, val_label = target[:split], target[split:]
print ("Done")
print ("  Ratio :", ratio)
print ("  Train_range :", 0, "-", split-1)
print ("  Val_range :", split, "-", len(seqs)-1)

Initializing global variables... Done
  Filepath set to ./logs/
Importing modules... 

Using TensorFlow backend.


Done
Reading data from disk... {'CA', 'FE', 'MG', 'ZN', 'CO', 'CU', 'NI', 'MN'}
Done
Loading dictionaries... Done
Performing cross validation split... Done
  Ratio : 0.9
  Train_range : 0 - 52385
  Val_range : 52386 - 58206


In [3]:
print (label_dict)

{'CU': 0, 'NI': 1, 'FE': 2, 'CO': 3, 'MN': 4, 'CA': 5, 'MG': 6, 'ZN': 7}


In [4]:
df.groupby('ligandId').count()

,structureChainId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100
ligandId,,,,,,,,,,,,
CA,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606
CO,759,759,759,759,759,759,759,759,759,759,759,759
CU,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143
FE,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989
MG,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336
MN,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688
NI,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073
ZN,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613


## Data Generator

- <font color=blue>FOFE Encoding</font>

In [ ]:
train_args = {'sequences': train_seqs,
              'labels': train_label,
              'translator': vocab_dic_fofe}
val_args = {'sequences': val_seqs,
            'labels': val_label,
            'translator': vocab_dic_fofe}
common_args = {'batch_size': 100,
               'input_shape': (800,),
               'label_shape': (8, ),
               'shuffle': True}

train_gen = modules.FOFEGenerator(**train_args, **common_args)
val_gen = modules.FOFEGenerator(**val_args, **common_args)

## Model
- <font color=blue>CNN</font>

In [ ]:
# ProtVec:100, One-hot:20, blosum62:20, property:7
dimension = 800
cutoff = 8

import tensorflow as tf
import time
import matplotlib.pyplot as plt
% matplotlib inline
np.random.seed(2017) 
from keras.models import Sequential, Model
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, Reshape, Embedding, Input
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
import numpy as np
import keras
from keras.models import Model, load_model
from keras.optimizers import Adam, SGD, RMSprop
# Visualization
from keras.utils import plot_model

input_shape = (dimension,)

input_0 = Input(shape=input_shape, dtype='float32')
input_0_reshape = Reshape((1,dimension,1), input_shape=(dimension,))(input_0)
conv2d_3 = Convolution2D(2, 1, 3, border_mode='same')(input_0_reshape)
conv2d_5 = Convolution2D(2, 1, 5, border_mode='same')(input_0_reshape)
conv2d_7 = Convolution2D(2, 1, 7, border_mode='same')(input_0_reshape)

x = keras.layers.concatenate([conv2d_3,conv2d_5,conv2d_7])
x = Activation('relu')(x)
x = Flatten()(x)
x = Dense(cutoff, activation='relu')(x)
output_0 = Dense(cutoff, activation='softmax')(x)
#output_0_reshape = Reshape((cutoff,1), input_shape=(cutoff,))(output_0)

#model = Model(inputs=input_0, outputs=output_0_reshape)
model = Model(inputs=input_0, outputs=output_0)                              
# end of the MODEL

sgd = SGD(lr = 0.01, momentum = 0.9, decay = 0, nesterov = False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [ ]:
model_args = {'model': model, 
              'generators': [train_gen, val_gen], 
              'callbacks': [], 
              'post_train_args': {'user': user, 
                                  'model': model_name, 
                                  'result': output_file, 
                                  'fig_path': fig_path}}

trainer = modules.Trainer(**model_args)

In [ ]:
import warnings; 
warnings.simplefilter('ignore')
trainer.start(epoch=15)

In [5]:
# later...
from keras.models import model_from_json
# load json and create model
json_file = open('./models/metal_predict.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("./models/metal_predict.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
train_args = {'sequences': seqs,
              'labels': target,
              'translator': vocab_dic_fofe}
common_args = {'batch_size': 1,
               'input_shape': (800,),
               'label_shape': (8, ),
               'shuffle': False}
train_gen = modules.FOFEGenerator(**train_args, **common_args)

In [7]:
metal_predictions = []
Y = []
# for s in seqs:
#     metal_predictions.append(modules.FOFE(s))
    

for i in range(len(train_gen)):
    x,y = train_gen[i]
    metal_predictions.append(model.predict(x))
    Y.append(y)
    

In [8]:
inv_label_dict = {v: k for k, v in label_dict.items()}
l1 = []
l2 = []
for i,j in enumerate(Y):
    max_i = max(j[0])
    x = [a for a, b in enumerate(Y[i][0]) if b == max_i]
    l2.append(inv_label_dict[x[0]])
    
for i,j in enumerate(metal_predictions):
    max_i = max(j[0])
    x = [a for a, b in enumerate(metal_predictions[i][0]) if b == max_i]
    l1.append(inv_label_dict[x[0]])

In [9]:
c = 0
for i in range(len(l1)):
    if l1[i] != l2[i]:
        c+=1
print ((len(l1)-c) / len(l1))

0.9160582060576906


In [10]:
df['metalPrediction'] = np.array(l1, dtype='O')

In [ ]:
df

In [11]:
df.to_parquet('Metal_all_20180601_predicted.parquet')

In [ ]:
df2 = pd.read_csv('Metal_all_20180601_predicted.csv', sep='\t')

In [ ]:
df2

In [ ]:
print (l1[:10])

In [ ]:
print (l2[:10])

In [ ]:
# Remove seqs containing 'U' and 'X'

duplicate_dict = {}
rows_to_delete = []
for i in range(seqs.shape[0]):
    if 'X' in seqs[i] \
    or 'U' in seqs[i] \
    or '3CO' in target[i]\
    or '3NI' in target[i] \
    or 'FE2'in target[i] \
    or 'CU1'in target[i]\
    or 'MN3' in target[i] \
    or np.isnan(cluster_numbers[i]):
        rows_to_delete.append(i)
        print (i, end=',')
    elif seqs[i] not in duplicate_dict.keys():
        duplicate_dict[seqs[i]] = target[i]
    else:
        if target[i] != duplicate_dict[seqs[i]]:
            rows_to_delete.append(i)
            print (i, end=',')
    
# df = df.drop(df.index[rows_to_delete])
# df.to_parquet('Metal_all_20180601.parquet')
seqs = np.delete(seqs, rows_to_delete, 0)
target = np.delete(target, rows_to_delete)
cluster_numbers = np.delete(cluster_numbers, rows_to_delete)